In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install recordlinkage

     |████████████████████████████████| 944 kB 417 kB/s eta 0:00:01


In [5]:
import recordlinkage

In [6]:
failed = pd.read_csv('../input/scrobbles-spreadsheets/Temp Sheets/Failed Results (Union minus Merged).csv')

failed_features = pd.read_csv('../input/scrobbles-spreadsheets/Temp Sheets/Failed Features.csv')

failed_not_features = pd.read_csv('../input/scrobbles-spreadsheets/Temp Sheets/Failed Scrobbles without Features.csv')

In [10]:
failed_not_features = failed_not_features.rename(columns={'track': 'song', 'artist': 'writer'})

In [7]:
indexer = recordlinkage.Index()
indexer.full()

<Index>

In [11]:
candidates = indexer.index(failed_features, failed_not_features)
print(len(candidates))

17080350


In [12]:
compare = recordlinkage.Compare()

In [14]:
compare.string('track',
            'song',
            threshold=0.85,
            label='tracks')

compare.string('artist',
            'writer',
            threshold = 0.85,
            label = 'artists')

<Compare>

In [15]:
features = compare.compute(candidates, failed_features, failed_not_features)

In [17]:
features.to_csv('features.csv', index=False)

In [18]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0         321
1.0       34224
0.0    17045805
dtype: int64

In [19]:
potential_matches = features[features.sum(axis=1) > 0].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'tracks':'artists'].sum(axis=1)

In [20]:
potential_matches

,level_0,level_1,tracks,artists,Score
0,0,0,1.0,0.0,1.0
1,0,1,1.0,0.0,1.0
2,0,2,1.0,0.0,1.0
3,0,9460,0.0,1.0,1.0
4,1,176,1.0,0.0,1.0
...,...,...,...,...,...
34540,1047,9460,0.0,1.0,1.0
34541,1047,16176,1.0,0.0,1.0
34542,1048,16204,0.0,1.0,1.0
34543,1049,9460,0.0,1.0,1.0


In [23]:
potential_matches = potential_matches.rename(columns={'level_0': 'features_idx', 'level_1': 'not_features_idx'})

In [24]:
potential_matches.to_csv('potential matches.csv', index=False)

In [25]:
failed_features['track_artist'] = failed_features[[
    'track', 'artist'
]].apply(lambda x: '_'.join(x), axis=1)

In [27]:
failed_not_features['song_writer'] = failed_not_features[['song', 'writer']].apply(lambda x: '_'.join(x), axis=1)

In [30]:
failed_not_features = failed_not_features.drop(['album', 'release_date', 'genres', 'length', 'popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness','liveness', 'valence','tempo','time_signature'], axis=1)

In [32]:
failed_not_features = failed_not_features.drop(['key', 'mode'], axis=1)

In [40]:
song_writer = failed_not_features['song_writer'].reset_index()
track_artist = failed_features['track_artist'].reset_index()

In [46]:
track_artist = track_artist.rename(columns={'index': 'features_idx'})
song_writer = song_writer.rename(columns={'index': 'not_features_idx'})

In [48]:
trackartist_merge = potential_matches.merge(track_artist, how='left')

In [49]:
trackartist_merge.head()

,features_idx,not_features_idx,tracks,artists,Score,track_artist
0,0,0,1.0,0.0,1.0,In Blue_Various Artists
1,0,1,1.0,0.0,1.0,In Blue_Various Artists
2,0,2,1.0,0.0,1.0,In Blue_Various Artists
3,0,9460,0.0,1.0,1.0,In Blue_Various Artists
4,1,176,1.0,0.0,1.0,See You Space Cowboys Not Final Mountain Root_...


In [54]:
final_merge = trackartist_merge.merge(song_writer, how='left')

In [55]:
final_merge

,features_idx,not_features_idx,tracks,artists,Score,track_artist,song_writer
0,0,0,1.0,0.0,1.0,In Blue_Various Artists,In Blue_Declan Mckenna
1,0,1,1.0,0.0,1.0,In Blue_Various Artists,In Blue_Declan Mckenna
2,0,2,1.0,0.0,1.0,In Blue_Various Artists,In Blue_Declan Mckenna
3,0,9460,0.0,1.0,1.0,In Blue_Various Artists,Pokémon_Various Artists
4,1,176,1.0,0.0,1.0,See You Space Cowboys Not Final Mountain Root_...,See You Space Cowboys Not Final Mountain Root_...
...,...,...,...,...,...,...,...
34540,1047,9460,0.0,1.0,1.0,Ragtime Cat (feat. Lilja Bloom)_Various Artists,Pokémon_Various Artists
34541,1047,16176,1.0,0.0,1.0,Ragtime Cat (feat. Lilja Bloom)_Various Artists,Ragtime Cat Feat. Lilja Bloom_Parov Stelar
34542,1048,16204,0.0,1.0,1.0,Nothing More (deaf Kev Remix)_Yoe Mase,Nothing More_Yoe Mase
34543,1049,9460,0.0,1.0,1.0,Mess_Various Artists,Pokémon_Various Artists


In [58]:
final_merge.sort_values(by=['Score'], ascending=False)

,features_idx,not_features_idx,tracks,artists,Score,track_artist,song_writer
8187,338,6885,1.0,1.0,2.0,Endless Summer - Instrumental_The Midnight,Endless Summer (instrumental)_The Midnight
30500,771,13233,1.0,1.0,2.0,Don’t Run_Mr Little Jeans,Don’t Run_Mr. Little Jeans
31922,865,15009,1.0,1.0,2.0,Evening In The Tavern_Pawel Blaszczak,Evening In The Tavern_Paweł Błaszczak
26766,664,12121,1.0,1.0,2.0,Holiday / Boulevard Of Broken Dreams_Green Day,Holiday/boulevard Of Broken Dreams_Green Day
22051,599,10775,1.0,1.0,2.0,Breathing Underwater - Ether Remix_Hollywood P...,Breathing Underwater (ether Remix)_Hollywood P...
...,...,...,...,...,...,...,...
11569,401,12766,0.0,1.0,1.0,Born And Raised - Reprise_John Mayer,I Don't Need No Doctor - Live At The Nokia The...
11568,401,12765,0.0,1.0,1.0,Born And Raised - Reprise_John Mayer,I Don't Need No Doctor - Live At The Nokia The...
11567,401,12764,0.0,1.0,1.0,Born And Raised - Reprise_John Mayer,I Don't Need No Doctor - Live At The Nokia The...
11566,401,12763,0.0,1.0,1.0,Born And Raised - Reprise_John Mayer,I Don't Need No Doctor - Live At The Nokia The...


In [57]:
final_merge.sort_values(by=['Score'], ascending=False).to_csv('Final Merge.csv', index=False)